In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[4]:


from Agent import Agent
from Game import Game
from Piece import *
import numpy as np
import random

#__pieces__ = ['Piece_Line_L']

__pieces__ = ['Piece_Dot','Piece_Diamond','Piece_Box_R','Piece_Box_L','Piece_Line_R','Piece_Line_L','Piece_Seven_RU', 'Piece_Seven_LU', 'Piece_Seven_RD', 'Piece_Seven_LU', 'Piece_Hook_RU', 'Piece_Hook_LU', 'Piece_Hook_RD', 'Piece_Hook_LD','Piece_T_RU', 'Piece_T_LU', 'Piece_T_RD', 'Piece_T_LD', 'Piece_Semi_U', 'Piece_Semi_D', 'Piece_Semi_RU', 'Piece_Semi_LU', 'Piece_Semi_RD', 'Piece_Semi_LD']
__score_for_move__=40
__score_for_clear__=20 # per title removed
#__reward_for_losing__=-10 # not used
#score_for_clear_bonus=1 # not sure how this works

class Controller(object):
    def __init__(self,lr,d):
        self.agent = Agent()
        #self.game = Game()
        self.score = 0
        self.lr = lr #0.1 # learning rate
        self.d = d #-0.1 # discount rate
        self.Qtable=dict()
    
    def play_games(self,n):
        for trial in range(1,n+1):
            print '******* TRIAL '+str(trial)+' *******'
            print "New Game"
            self.game = Game()
            print self.game
            
            i=1
            while True: # to test i<200, should be while(True)
                print '** Move ',str(i),': **'
                grid_str = self.game.get_grid()
                self.add_grid_to_Qtable(grid_str)

                i=i+1
                possible_actions=[]
                actions = self.get_available_pieces()
                for action in actions:
                    # check if can_put in any location
                    klass = globals()[action]
                    p = klass()
                    for l0 in range(5):
                        for l1 in range(5+l0):
                            if p.can_put(self.game,(l0,l1)):
                                possible_actions.append((action,(l0,l1)))
                    for l0 in range(5,9):
                        for l1 in range(13-l0):
                            if p.can_put(self.game,(l0,l1)):
                                possible_actions.append((action,(l0,l1)))

                if len(possible_actions)==0:
                    print 'Here'
                    trail = n # not sure if this is necessary, want to break from outer loop too
                    return self.exit() 
                    break
                else:
                    action = possible_actions[np.random.choice(len(possible_actions),1)[0]] # done this because np.choice doesn't work with list of tuples
                    print action
                    print action[0]
                    print action[1]
                    self.agent.move(action[0],self.game,action[1]) 
                    move_reward=0
                    self.score=self.score+__score_for_move__
                    move_reward=move_reward+__score_for_move__
                    # get h and d's that are now full and set them to 0
                    is_full_result = self.game.is_full()
                    fulls = is_full_result[0]
                    self.score = self.score + is_full_result[1]*__score_for_clear__
                    move_reward=move_reward+is_full_result[1]*__score_for_clear__
                    # make full null
                    for k, vs in fulls.items():
                        for v in vs:
                            getattr(self.game, k)(v,0)
                            #print 'cleared: ', (k,v)
                    print self.game
                    print 'current score: ',self.score
                    self.add_action_reward_to_Qtable(grid_str,action,move_reward)
                    #print self.Qtable
                
                
    def play_test(self):
        print "New Game"
        print self.game
        i=1
        while True: # to test i<200, should be while(True)
            print '****** Move ',str(i),': ******'
            i=i+1
            actions = self.get_available_pieces()
            # TODO agent strategy
            if(not self.agent.make_move(self.game,actions)):
                # no moves available # this should be checked by game not agent
                self.exit()
                break
            else:
                # if move available, move has been made in if statement
                self.score=self.score+__score_for_move__
                # get h and d's that are now full and set them to 0
                is_full_result = self.game.is_full()
                fulls = is_full_result[0]
                self.score = self.score + is_full_result[1]*__score_for_clear__
                # make full null
                for k, vs in fulls.items():
                    for v in vs:
                        getattr(self.game, k)(v,0)
                        print 'cleared: ', (k,v)
                print self.game
                print 'current score: ',self.score
            
    def exit(self):
        print 'End of Game'
        print self.Qtable
    
    def get_available_pieces(self):
        pieces = np.random.choice(__pieces__,3)
        for piece in pieces:
            klass = globals()[piece]
            p = klass()
            print p
            print '___'
        return pieces
    def add_grid_to_Qtable(self,grid_str):
        if not grid_str in self.Qtable:
            self.Qtable[grid_str]={}
            return True
        return False
        # if Q value for an action dne, randomly initialize it and use it.
    
    def max_expQval(self,grid_str):
        max_= 0
        for k,v in self.Qtable[grid_str].iteritems():
            if v>max_:
                max_=v
        return max_
    
    def add_action_reward_to_Qtable(self,grid_str,action_str,reward):
        # at this point grid_str is definitely in Qtable
        if action_str in self.Qtable[grid_str]:
            current_Qval = self.Qtable[grid_str][action_str]
            self.Qtable[grid_str][action_str] = current_Qval + self.lr*(reward+self.d*self.max_expQval(grid_str)-current_Qval)
        else:
            rand = random.random()
            self.Qtable[grid_str][action_str] = rand
            self.Qtable[grid_str][action_str] = rand + self.lr*(reward+self.d*self.max_expQval(grid_str)-rand)
            
    
'''        
controller = Controller(0.1,-0.1)
grid_str = controller.game.get_grid()
controller.add_grid_to_Qtable(grid_str)
print controller.Qtable
p = Piece_Line_H()
controller.add_action_reward_to_Qtable(grid_str,p.get_name(),10)
print controller.Qtable
p = Piece_Line_H()
controller.add_action_reward_to_Qtable(grid_str,p.get_name(),20)
print controller.Qtable
controller.add_action_reward_to_Qtable(grid_str,p.get_name(),0)
print controller.Qtable
p2 = Piece_Line_R()
controller.add_action_reward_to_Qtable(grid_str,p2.get_name(),0)
print controller.Qtable
'''

'''
def learning(self, max_games)
    game_count = 0
    while game_count < maxGames  
        current_grid = game.get_grid()
        if current_grid not in table
            Q.addKey(current_grid)
            action = chooseAction(stateKey, table)
    nextState = execute(action)
    nextKey = makeKey(nextState, opponent(player))
    reward = reinforcement(nextState)
    if nextKey not in table
       addKey(nextKey)
    updateQvalues(stateKey, action, nextKey, reward)
    if game over
       reset game
       state = startState
       games += 1
    else
       state = nextState
       switchPlayers()
'''

controller = Controller(0.1,-0.1)
controller.play_games(1)



******* TRIAL 1 *******
New Game
    [0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0]
  [0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0]
  [0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0]
    [0, 0, 0, 0, 0]

** Move  1 : **
 o 
o o
 o
___
o
___
o
 o
o o
___
('Piece_Dot', (7, 5))
Piece_Dot
(7, 5)
moved:
o  at  (7, 5)
    [0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0]
  [0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0]
  [0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 1]
    [0, 0, 0, 0, 0]

current score:  40
** Move  2 : **
o
___
o o o
   o
___
o
___
('Piece_Dot', (6, 1))
Piece_Dot
(6, 1)
moved:
o  at  (6, 1)
    [0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0]
  [0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0]
  [0, 1, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 1]
    [0, 0, 0, 0, 0]

current score:  80
** Move  3 : **
o o
 o
  o
___
 o o
o   o
___
  o
 o o
o
___
('Piece_Semi_D',